## 15-puzzle solver

##Basic idea

First, we should decide how to characterize states: For this I will write the three rows one by one. And to depict the empty space I will use 16.

Now for further analysis, as explained in the Medium post, it would be best to solve the puzzle row wise. First we can solve the first row, then the second row and then we can solve the third and 4th rows together. Otherwise, the number of states would be too large to do anything. Breaking tasks in this way reduces the number of states we need to deal with at a time greatly.
However it is worth noting that when we do this, we lose the optimality of the solution. That is, a method involving lower number of steps might be possible.

Now, building up the further high level idea, for each of the 3 parts, we should be able to model the MDP without too much difficulty. On this we can apply value iteration method and build a near optimal policy. Then we will take as input the puzzle to be solved, and one by one apply the three policies on it and hence end up solving it.

In [2]:
import numpy as np
from itertools import permutations


def printpuzzle(a):
    for i in range(4): 
        print(" | ".join(f"{a[j]:>3}" for j in range(4 * i, 4 * i + 4)))
        if i < 3: 
            print("---+-----+-----+---")
    print()

In [3]:
def findindex(a): #function for finding which element is 16 in a list or tuple
    for i in range(0,16):
        if (a[i]==16):
            return i
def moveup(a, part): #function that takes in a state tuple and returns the state tuple after the empty space is moved upward
    b=list(a)
    index=findindex(b)
    if (index<4*(1+part)):
        return tuple(b)
    else:
        b[index]=b[index-4]
        b[index-4]=16
        return tuple(b)
def movedown(a, part): #function that takes in a state tuple and returns the state tuple after the empty space is moved downward
    b=list(a)
    index=findindex(b)
    if (index>=(12)):
        return tuple(b)
    else:
        b[index]=b[index+4]
        b[index+4]=16
        return tuple(b)
def moveleft(a): #function that takes in a state tuple and returns the state tuple after the empty space is moved leftward
    b=list(a)
    index=findindex(b)
    if ((index%4)==0):
        return tuple(b)
    else:
        b[index]=b[index-1]
        b[index-1]=16
        return tuple(b)
def moveright(a): #function that takes in a state tuple and returns the state tuple after the empty space is moved rightward
    b=list(a)
    index=findindex(b)
    if ((index%4)==3):
        return tuple(b)
    else:
        b[index]=b[index+1]
        b[index+1]=16
        return tuple(b)
def move(s, a, which_part):
    if (s=="up"):
        return moveup(a, which_part)
    elif (s=="down"):
        return movedown(a, which_part)
    elif (s=="left"):
        return moveleft(a)
    else:
        return moveright(a)


In [4]:
actions=["up","down","left","right"]

In [5]:
def terminality(which_part):
    if (which_part==0):
        return lambda a: (a[0]==1 and a[1]==2 and a[2]==3 and a[3]==4)
    elif (which_part==1): 
        return lambda a: (a[4]==5 and a[5]==6 and a[6]==7 and a[7]==8)
    else:
        return lambda a: (a[8]==9 and a[9]==10 and a[10]==11 and a[11]==12 and a[12]==13 and a[13]==14 and a[14]==15 and a[15]==16)

def presence(which_part):
    if (which_part==0):
        return lambda a: ((1<=a and a<=4) or a==16)
    elif (which_part==1):
        return lambda a: ((5<=a and a<=8) or a==16)
    else:
        return lambda a: ((9<=a and a<=16))

In [6]:
states0=[]
tups = list(permutations(range(16), 5))
for tup in tups:
    eqstate = [0]*16
    for i in range(4):
        eqstate[tup[i]]=1+i
    eqstate[tup[4]]=16
    states0.append(tuple(eqstate))
states1=[]
tups = list(permutations(range(4,16), 5))
for tup in tups:
    eqstate = [0]*16
    for i in range(4):
        eqstate[tup[i]]=5+i
    eqstate[tup[4]]=16
    states1.append(tuple(eqstate))
states2=[]
tups = list(permutations(range(8,16), 8))
for tup in tups:
    eqstate = [0]*16
    for i in range(8):
        eqstate[tup[i]]=9+i
    states2.append(tuple(eqstate))

#some statements to help confirm that the states were made properly
print(len(states0))
print(len(states1))
print(len(states2))

524160
95040
40320


In [7]:
#creating mdp
def createmdp(states, which_part):
    mdp={}
    isterminal=terminality(which_part)
    for state in states:
        transitions={}
        alr_terminal = isterminal(state)
        for action in actions:
            if (alr_terminal):
                transitions[action]=[(state,0,True)]
            else:
                new_state=move(action,state, which_part)
                if(new_state==state):
                    continue
                if (isterminal(new_state)):
                    transitions[action]=[(tuple(new_state),1, True)]
                    print("chalo milgaya") #debugging statement, should get this 10 times
                else:
                    transitions[action]=[(tuple(new_state),0, False)]
        mdp[state]=transitions
    return mdp

mdp0 = createmdp(states0, 0)
mdp1 = createmdp(states1, 1)
mdp2 = createmdp(states2, 2)

chalo milgaya
chalo milgaya
chalo milgaya
chalo milgaya
chalo milgaya
chalo milgaya
chalo milgaya
chalo milgaya
chalo milgaya
chalo milgaya


In [10]:
def createpolicy(states, mdp, part):
    theta=0.0001
    delta=69
    values = {}
    pi = {}
    gamma=0.9
    isterminal=terminality(part)
    while (delta>theta):
        delta=0
        for state in states:
            if (isterminal(state)):
                values[state]=0
                continue
            v = values.get(state, 0)
            values[state]=v
            m = float('-inf')
            for action in mdp[state]:
                new_value=0
                for case in mdp[state][action]:
                    new_value+=((case[1]+(gamma*values.get(case[0],0))))
                if (new_value>m):
                    m=new_value
                    pi[state]=action
            values[state]=m
            delta=max(delta,abs(v-m))
    return pi
policy0=createpolicy(states0,mdp0,0)
policy1=createpolicy(states1,mdp1,1)
policy2=createpolicy(states2,mdp2,2)

In [11]:
def solve(pi,which_part,inp):
    isterminal=terminality(which_part)
    pres=presence(which_part)
    perf=[0]*16
    for i in range(16):
        if(pres(inp[i])):
            perf[i]=inp[i]
    while(not isterminal(perf)):
        perform=tuple(perf)
        act=pi[perform]
        perf=move(act,perform,which_part)
        inp=list(move(act,inp,which_part))
        printpuzzle(inp)
    return inp

In [ ]:
#to check from a particular case
inp=[7,8,4,5,3,12,15,1,14,13,2,9,10,11,16,6]
printpuzzle(inp)
ina=solve(policy0,0,inp)
inb=solve(policy1,1,ina)
inc=solve(policy2,2,inb)

In [12]:
#incase you wanna send an input and try urself.
inp=[0]*16
for i in range(16):
   inp[i]=int(input())
printpuzzle(inp)
ina=solve(policy0,0,inp)
inb=solve(policy1,1,ina)
inc=solve(policy2,2,inb)

  7 |   8 |   4 |   5
---+-----+-----+---
  3 |  12 |  15 |   1
---+-----+-----+---
 14 |  13 |   2 |   9
---+-----+-----+---
 10 |  11 |  16 |   6

  7 |   8 |   4 |   5
---+-----+-----+---
  3 |  12 |  15 |   1
---+-----+-----+---
 14 |  13 |   2 |   9
---+-----+-----+---
 10 |  16 |  11 |   6

  7 |   8 |   4 |   5
---+-----+-----+---
  3 |  12 |  15 |   1
---+-----+-----+---
 14 |  16 |   2 |   9
---+-----+-----+---
 10 |  13 |  11 |   6

  7 |   8 |   4 |   5
---+-----+-----+---
  3 |  16 |  15 |   1
---+-----+-----+---
 14 |  12 |   2 |   9
---+-----+-----+---
 10 |  13 |  11 |   6

  7 |   8 |   4 |   5
---+-----+-----+---
  3 |  15 |  16 |   1
---+-----+-----+---
 14 |  12 |   2 |   9
---+-----+-----+---
 10 |  13 |  11 |   6

  7 |   8 |   4 |   5
---+-----+-----+---
  3 |  15 |   1 |  16
---+-----+-----+---
 14 |  12 |   2 |   9
---+-----+-----+---
 10 |  13 |  11 |   6

  7 |   8 |   4 |  16
---+-----+-----+---
  3 |  15 |   1 |   5
---+-----+-----+---
 14 |  12 |   2 |   9
